In [ ]:
import cv2
import mediapipe as mp
from flask import Flask, Response
import pandas as pd
import time
from threading import Thread

app = Flask(__name__)

# 전역 변수로 웹캠 초기화
cap = cv2.VideoCapture(0)

# 전역 변수로 Mediapipe Holistic 모델 초기화
holistic = mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 초기 DataFrame 생성
df = pd.DataFrame()

# 웹캠 캡처를 별도 스레드에서 실행
def capture_frames():
    global cap, df
    with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            # 이미지 좌우 반전
            image = cv2.flip(image, 1)

            # 이미지 처리
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            # 이미지에 포즈 그리기
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.pose_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp.solutions.pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp.solutions.drawing_styles.get_default_pose_landmarks_style())

                # 각 랜드마크의 인덱스 번호 그리기 및 데이터 수집
                detected_landmarks = len(results.pose_landmarks.landmark)
                row = []
                for idx, landmark in enumerate(results.pose_landmarks.landmark):
                    if idx >= detected_landmarks:
                        break

                    # 랜드마크의 화면 좌표 변환 및 데이터 수집
                    x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
                    cv2.putText(image, str(idx), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    row.extend([landmark.x, landmark.y, landmark.z])

                # 첫 번째 프레임에서 DataFrame 컬럼 생성
                if df.empty:
                    columns = []
                    for i in range(detected_landmarks):
                        columns.extend([f'x_{i}', f'y_{i}', f'z_{i}'])
                    df = pd.DataFrame(columns=columns)

                # 데이터프레임에 랜드마크 데이터 추가
                tmp = pd.DataFrame([row], columns=df.columns)
                df = pd.concat([df, tmp])

            # 이미지 표시
            cv2.imshow('MediaPipe Pose', image)  # 이미지 좌우 반전 후 표시
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

# 웹캠 캡처 스레드 시작
capture_thread = Thread(target=capture_frames)
capture_thread.start()

def generate_frames():
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        ret, buffer = cv2.imencode('.jpg', frame)
        if not ret:
            break
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route("/")
def bye():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    app.run(host="192.168.0.98", port=9000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.0.98:9000
Press CTRL+C to quit
192.168.0.98 - - [07/Nov/2023 11:15:10] "GET / HTTP/1.1" 200 -
